In [1]:
import sys
import os
from dotenv import load_dotenv

load_dotenv(os.path.expanduser('~/.env'), verbose=True)

data_dir = os.getenv('DATA_IGN_DIR')
pretrained_model_dir = os.getenv('PRETRAINED_MODEL_DIR')

sys.path.insert(0, '..')

In [2]:
import logging
os.environ["CUDA_VISIBLE_DEVICES"] = '0'
import random
from dataclasses import dataclass, field
from typing import Optional, List

import datasets
import numpy as np
from datasets import load_dataset, concatenate_datasets

from pprint import pprint

import evaluate
import transformers
from transformers import (
    GPT2Tokenizer,
    GPT2LMHeadModel,
    DataCollatorWithPadding,
    EvalPrediction,
    PretrainedConfig,
    Trainer,
    TrainingArguments,
    default_data_collator,
    set_seed,
    get_scheduler,
)

from torch.utils.data import DataLoader
import torch

from pdb import set_trace

from torch.nn import CrossEntropyLoss, MSELoss

from transformers.trainer_utils import EvalLoopOutput

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, recall_score

from tqdm import tqdm
import json
from datetime import datetime
import random
from datasets import concatenate_datasets

from transformers import EarlyStoppingCallback

import torch.nn as nn
import torch.nn.functional as F
from st_moe_distilgpt2_perplexity import STMoE_DistilGPT2

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device_count = torch.cuda.device_count()
print(device, device_count)

task_to_keys = {
    "cola": ("sentence", None),
    "mnli": ("premise", "hypothesis"),
    "mrpc": ("sentence1", "sentence2"),
    "qnli": ("question", "sentence"),
    "qqp": ("question1", "question2"),
    "rte": ("sentence1", "sentence2"),
    "sst2": ("sentence", None),
    "stsb": ("sentence1", "sentence2"),
    "wnli": ("sentence1", "sentence2"),

    'rotten_tomatoes': ("text", None),
    'imdb': ("text", None),
    'yelp_polarity': ("text", None),
    
}


eval_data_dict = {'imdb': 'test', 'yelp_polarity': 'test'}

is_glue = {"cola": True,
            "mnli": True,
            "mrpc": True,
            "qnli": True,
             "qqp": True,
             "rte": True,
            "sst2": True,
            "stsb": True,
            "wnli": True,}

current_time = datetime.now().strftime('%Y%m%d-%H%M%S')

cuda 1


In [3]:
checkpoint_name = 'checkpoint_4.pt'
checkpoint_path = os.path.join(pretrained_model_dir, checkpoint_name)

In [4]:
tokenizer = GPT2Tokenizer.from_pretrained('distilGPT2')
model = STMoE_DistilGPT2()
model.load_state_dict(torch.load(checkpoint_path, map_location=device))

print(model.net.transformer.config.vocab_size)

model.net.config.pad_token_id = None
model.net.resize_token_embeddings(len(tokenizer))

print(model.net.transformer.config.vocab_size)

Using pad_token, but it is not set yet.


50258
50257


In [6]:
model.net.dtype

torch.float32

In [5]:
test = load_dataset("wikitext", "wikitext-2-raw-v1", split="test")
encodings = tokenizer("\n\n".join(test["text"]), return_tensors="pt")

In [6]:
model = model.to(device)
model.eval()

max_length = model.net.config.n_positions
stride = 1024
seq_len = encodings.input_ids.size(1)

loss_fct = CrossEntropyLoss()

nlls = []
prev_end_loc = 0
for begin_loc in tqdm(range(0, seq_len, stride)):
    end_loc = min(begin_loc + max_length, seq_len)
    trg_len = end_loc - prev_end_loc  # 마지막 루프의 스트라이드 값과 다를 수 있음
    
    input_ids = encodings.input_ids[:, begin_loc:end_loc].to(device)

    target_ids = input_ids.clone()
    target_ids[:, :-trg_len] = -100

    with torch.no_grad():
        outputs = model(input_ids=input_ids)

        lm_logits = outputs[0]
        
        # Shift so that tokens < n predict n
        shift_logits = lm_logits[..., :-1, :].contiguous()
        shift_labels = target_ids[..., 1:].contiguous()
        # Flatten the tokens
        loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))

    nlls.append(loss)

    prev_end_loc = end_loc
    if end_loc == seq_len:
        break

ppl = torch.exp(torch.stack(nlls).mean())
print(ppl)

100%|█████████▉| 280/281 [00:06<00:00, 40.63it/s]

tensor(112195.5234, device='cuda:0')
